## Simulated Annealing Code

This code implements **Simulated Annealing** (SA), a probabilistic technique used for approximating the global optimum of a function. The idea is inspired by the process of annealing in metallurgy, where a material is heated and then slowly cooled to remove defects and optimize the structure. In optimization, this translates to exploring the solution space broadly at high "temperatures" and then converging to a minimum as the temperature decreases.

---

### 1. **Basics**

The **annealing** process in optimization consists of the following key elements:

- **State**: A candidate solution to the optimization problem. In your case, this is represented by the variable `state`, which is initialized with a random value within a bounded range.
  
- **Energy**: A function that measures the quality of the current state. This corresponds to the objective function you're trying to minimize. In this implementation, `energy()` represents the value of the objective function at the current state.

- **Move**: A small random change in the state that explores neighboring solutions. This is achieved by perturbing the current state slightly in `move()`.

- **Temperature (T)**: A control parameter that starts high and gradually decreases. It governs the likelihood of accepting worse solutions as the system explores the solution space.

- **Acceptance Criterion**: At each step, you accept the new state if the new energy is lower than the current energy. If the new energy is worse, the new state is accepted with a probability based on the temperature and the energy difference, determined by the **Metropolis criterion**:

$$
\text{acceptance probability} = \exp\left(\frac{-\Delta E}{T}\right)
$$

where $ \Delta E $ is the change in energy and $ T $ is the current temperature.

---

### 2. **Mathematics of the Annealing Process**

#### 2.1 **Energy Function**

In the `FunctionAnnealer` class, the energy function represents the objective function $ f(x) $ to minimize. It is defined as:

$$
E(\text{state}) = f(x) \quad \quad \text{where } x = \text{state}[0]
$$

The function you're minimizing is passed as an argument and depends on the specific problem. Three example functions are defined in the `functions` array and chosen interactively by the user.

#### 2.2 **Move Operation**

In each step, the current state is perturbed slightly in `move()`, where the state $ x $ is updated by adding a small random number:

$$
\text{new } x = x + u(-0.1, 0.1)
$$

Here, $ u(-0.1, 0.1) $ is a random uniform value between -0.1 and 0.1, meaning that the state moves to a nearby solution in the search space. This represents the exploration of the solution space.

The state $ x $ is also bounded between -4 and 4 to prevent the search from diverging too far.

**Move Size**: A key consideration in the move operation is the step size of perturbations. Larger moves allow for more aggressive exploration of the solution space, helping the algorithm escape local minima. However, as the temperature decreases, smaller move sizes help fine-tune the solution and improve convergence near a global minimum.

#### 2.3 **Metropolis Criterion**

The Metropolis criterion is the core mechanism of Simulated Annealing, which allows the algorithm to accept "worse" solutions under certain conditions to escape local minima. It is mathematically defined as:

$$
P(\text{accept}) =
\begin{cases}
1, & \text{if } \Delta E < 0 \\
\exp\left(-\frac{\Delta E}{T}\right), & \text{if } \Delta E \geq 0
\end{cases}
$$

where $ \Delta E = E_{\text{new}} - E_{\text{current}} $ is the change in energy and $ T $ is the current temperature.

- If $ \Delta E < 0 $, i.e., the new solution is better, it is always accepted.
- If $ \Delta E \geq 0 $, i.e., the new solution is worse, it is accepted with a probability proportional to $ \exp\left(-\frac{\Delta E}{T}\right) $, which decreases as the temperature $ T $ decreases.

This acceptance mechanism is crucial for allowing the algorithm to escape **local minima** and search for better solutions in the global landscape.

#### 2.4 **Temperature Schedule**

The temperature is initialized at $ T_{\text{max}} $ and decreases over time using an exponential cooling schedule:

$$
T_{\text{new}} = \max(T_{\text{min}}, 0.99 \cdot T_{\text{current}})
$$

This ensures that the temperature decreases at each step, gradually reducing the probability of accepting worse solutions as the algorithm progresses.

##### **Alternative Temperature Schedules**:
In practice, various cooling schedules can be applied. Some of the common ones include:

1. **Linear Cooling**:
   $$
   T_{\text{new}} = T_{\text{current}} - \Delta T
   $$
   where $ \Delta T $ is a constant decrement.
   
2. **Logarithmic Cooling**:
   $$
   T_{\text{new}} = \frac{T_{\text{max}}}{\log(k + 1)}
   $$
   where $ k $ is the current iteration step.
   
3. **Geometric Cooling** (Exponential):
   $$
   T_{\text{new}} = \alpha \cdot T_{\text{current}}
   $$
   where $ \alpha $ is a constant multiplier (e.g., 0.99).

Each of these schedules influences how quickly the algorithm reduces its exploration and transitions to fine-tuning the solution.

#### 2.5 **Stopping Criteria**

Simulated Annealing can stop based on several conditions:

1. **Temperature Threshold**: The algorithm stops when the temperature reaches $ T_{\text{min}} $, which defines the point at which further exploration is no longer worthwhile.

2. **Fixed Number of Iterations**: The algorithm may run for a pre-defined number of iterations, regardless of temperature.

3. **Energy Convergence**: If the energy change over successive iterations is below a certain threshold, the algorithm may terminate early, assuming that it has converged to an optimal solution.

---

### 3. **Execution of the Annealing Process**

In the `anneal()` function, the simulated annealing algorithm follows these steps:

1. **Initialize** the system:
   - Set the initial temperature $ T = T_{\text{max}} $.
   - Initialize the current state and calculate its energy.

2. **Iterate** over a fixed number of steps:
   - At each step, save the current state and energy.
   - Use the `move()` function to generate a new candidate state by perturbing the current state.
   - Compute the energy of the new state.
   - If the new energy is better, accept the new state. If the new energy is worse, accept it with a probability based on the Metropolis criterion.
   - Record the best solution encountered.
   - Decrease the temperature according to the cooling schedule.
   - Update the energy and temperature histories for plotting.

3. **Return** the best state and energy found during the search, as well as the history of energies and temperatures for analysis.

---

### 4. **Objective Functions**

The three objective functions provided are non-trivial and likely have multiple local minima, making them good candidates for testing simulated annealing. They are:

1. $f_1(x) = e^{-0.4x} \sin(20x) + \cos(5x) \sin(10x)$
   
   This function oscillates rapidly, creating multiple local minima, and decays exponentially, favoring solutions closer to 0.

2. $f_2(x) = \frac{\sin(15x) + \cos(7x)}{x^2 + 1} + \frac{\sin(5x)}{x + 2}$

   This function combines trigonometric oscillations with rational terms, leading to sharp peaks and valleys in the solution space.

3. $ f_3(x) = \sin(10x) \cdot e^{\sin(5x)} + \cos(20x) \cdot e^{\cos(3x)}$

   This function is a product of exponentials and trigonometric terms, creating a complex landscape with many local minima.


In [ ]:
import os
#from simanneal import Annealer
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
from matplotlib_venn import venn3
from sklearn.decomposition import PCA
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.metrics import silhouette_samples, silhouette_score, pairwise
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from matplotlib.animation import FuncAnimation

import math
import ipywidgets as widgets
import random
from IPython.display import display, clear_output

In [ ]:
def f1(x):
    return np.exp(-0.4 * x) * np.sin(20 * x) + np.cos(5 * x) * np.sin(10 * x)

def f2(x):
    return (np.sin(15 * x) + np.cos(7 * x)) / (x**2 + 1) + np.sin(5 * x) / (x + 2)

def f3(x):
    return np.sin(10 * x) * np.exp(np.sin(5 * x)) + np.cos(20 * x) * np.exp(np.cos(3 * x))

In [ ]:
def simulated_annealing(func, x0, temperature, cooling_rate, step_size, iterations):
    current_x = x0
    current_value = func(current_x)
    best_x = current_x
    best_value = current_value
    history = [(current_x, current_value)]

    for i in range(iterations):
        candidate_x = current_x + step_size * (random.uniform(-1, 1))
        candidate_x = max(min(candidate_x, 4), -4)
        candidate_value = func(candidate_x)

        delta = candidate_value - current_value
        acceptance_probability = math.exp(-delta / temperature) if delta > 0 else 1.0

        if random.random() < acceptance_probability:
            current_x = candidate_x
            current_value = candidate_value

        if current_value < best_value:
            best_x = current_x
            best_value = current_value

        temperature *= cooling_rate

        history.append((current_x, current_value))

    return best_x, best_value, history

In [ ]:
def plot_functions_and_annealing(func_choice, x0, temperature, cooling_rate, step_size, iterations):
    if func_choice == 1:
        func = f1
        func_name = "f1(x)"
    elif func_choice == 2:
        func = f2
        func_name = "f2(x)"
    elif func_choice == 3:
        func = f3
        func_name = "f3(x)"
    else:
        return

    best_x, best_value, history = simulated_annealing(func, x0, temperature, cooling_rate, step_size, iterations)

    x_values = np.linspace(-4, 4, 1000)
    y_values = [func(x) for x in x_values]

    plt.figure(figsize=(10, 6))
    plt.plot(x_values, y_values, label=f"{func_name}", color='blue')
    plt.scatter([h[0] for h in history], [h[1] for h in history], color='red', s=10, label='Annealing History')
    plt.scatter([best_x], [best_value], color='green', s=100, label=f'Best x: {best_x:.2f}, Best value: {best_value:.2f}')
    plt.title(f"Simulated Annealing on {func_name}")
    plt.xlabel("x")
    plt.ylabel("f(x)")
    plt.xlim([-4, 4])
    plt.ylim([-6, 6])
    plt.legend()
    plt.grid(True)
    plt.show()

func_choice_widget = widgets.Dropdown(
    options=[('f1(x)', 1), ('f2(x)', 2), ('f3(x)', 3)],
    value=1,
    description='Function:',
)

x0_widget = widgets.FloatSlider(
    value=0.0,
    min=-4.0,
    max=4.0,
    step=0.1,
    description='Initial x:'
)

temperature_widget = widgets.FloatSlider(
    value=10.0,
    min=0.1,
    max=50.0,
    step=0.1,
    description='Temp:'
)

cooling_rate_widget = widgets.FloatSlider(
    value=0.95,
    min=0.8,
    max=0.999,
    step=0.001,
    description='Cooling rate:'
)

step_size_widget = widgets.FloatSlider(
    value=0.5,
    min=0.01,
    max=1.0,
    step=0.01,
    description='Step size:'
)

iterations_widget = widgets.IntSlider(
    value=1000,
    min=100,
    max=5000,
    step=100,
    description='Iterations:'
)

interactive_plot = widgets.interactive(
    plot_functions_and_annealing,
    func_choice=func_choice_widget,
    x0=x0_widget,
    temperature=temperature_widget,
    cooling_rate=cooling_rate_widget,
    step_size=step_size_widget,
    iterations=iterations_widget,
)

In [ ]:
display(interactive_plot)

interactive(children=(Dropdown(description='Function:', options=(('f1(x)', 1), ('f2(x)', 2), ('f3(x)', 3)), va…